<a href="https://colab.research.google.com/github/hotdog1029/deeplearning/blob/main/0_1_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EA%B8%B0%EB%B3%B8(Autograd).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autograd
파이토치를 이용해 코드를 작성할 때 Back Propagation을 이용해 파라미터를 업데이트하는 방법은 Autograd 방식으로 쉽게 구현할 수 있도록 설정돼 있다.

In [ ]:
import torch

if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

우선, 파이토치를 이용하기 위해 torch를 임포트한다. torch의 torch.cuda.is_available()은 현재 파이썬이 실행되고 있는 환경에서 torch module를 이용할 때 GPU를 이용해 계산할 수 있는지를 파악하는 메서드이다. 위 파이썬 코드를 보면 if문을 이용해 torch.cuda.is_available()이 참이면 'cuda' 장비를 이용하는 것으로 설정하고 거짓이면 cpu 장비를 이용해 계산한다는 것을 의미한다.

In [ ]:
BATCH_SIZE = 64
INPUT_SIZE = 1000
HIDDEN_SIZE = 100
OUTPUT_SIZE = 10

BATCH_SIZE는 딥러닝 모델에서 파라미터를 업데이트할 때 계산되는 데이터의 개수 즉, BATCH_SIZE 수만큼 데이터를 이용해 Output을 계산하고 BATCH_SIZE 수만큼 출력된 결과값에 대한 오차값을 계산한다. BATCH_SIZE 수만큼 계산된 오차값을 평균해 Back propagation을 적용하고 이를 바탕으로 파라미터를 업데이트한다. 이 예제에서 이용하는 BATCH_SIZE는 64이며 이는 Input으로 이용되는 데이터가 64개라는 것을 의미
INPUT_SIZE는 딥러닝 모델에서의 Input의 크기이자 입력층의 노드 수를 의미한다. 앞의 BACTH_SIZE와 혼동하면 안 된다. INPUT_SIZE는 딥러닝 모델에서의 입력층의 노드 수를 의미하며 이 예제에서는 1000이므로 입력 데이터의 크기가 1000이라는 것을 의미한다. 즉, 1000의 크기의 벡터 값을 의미한다. BATCH_SIZE가 64이므로 1000의 크기의 벡터 값을 64개 이용한다는 것을 의미
HIDDEN_SIZE는 딥러닝 모델에서 Input을 다수의 파라미터를 이용해 결과에 한 번 더 계산되는 파라미터 수를 의미, 즉 입력층에서 은닉층으로 전달됐을 때 은닉층의 노드 수를 의미한다. 이 에제에서는 (64,1000)의 Input들이 (1000,100)의 크기의 행렬과 행렬 곱을 계산하기 위해 설정한 수 이다.
OUTPUT_SIZE는 딥러닝 모델에서 최종으로 출력되는 값의 벡터의 크기를 의미한다. 보통 Output의 크기는 최종으로 비교하고자 하는 레이블의 크기와 동일하게 설정한다.

In [ ]:
x = torch.randn(BATCH_SIZE,  # (1)
                INPUT_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = False)
y = torch.randn(BATCH_SIZE,    # (2)
                OUTPUT_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = False)
w1 = torch.randn(INPUT_SIZE,   # (3)
                HIDDEN_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = True)
w2 = torch.randn(HIDDEN_SIZE,   # (4)
                OUTPUT_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = True)

randn 은 평균이 0, 표준편차가 1인 정규분포에서 샘플링 한 값으로, 데이터를 
만든다는 것을 의미한다.

(1) 크기가 1000짜리의 벡터를 64개 만들기 위해 BATCH_SIZE는 '64', INPUT_SIZE는 1000으로 설정했으며 x는 (64,1000) 모양의 데이터가 생성된다. 이때 생성된 데이터는 미리 설정한 DEVICE를 이용해 계산할 것이기 때문에 device = DEVICE로 맞춰주며 데이터 형태는 float으로 설정한다. 또한 해당 데이터는 Input으로 이용되기 때문에 Gradient를 계산할 필요가 없다. 우리는 파라미터 값을 업데이트하기 위해 Gradient를 계산하는 것이지 Input에 대해 Gradient를 하는 것이 아니다. 따라서 requires_grad = False 로 설정한다.

(2) Output도 Input을 설정하는 것과 동일하다. Output 역시 BATCH_SIZE 수만큼 결과값이 필요하다.

(3) w1은 input의 데이터 크기가 1000이며 이와 행렬 곱을 하기위해 다음 행의 값이 1000이어여 한다. 또한 행렬 곱을 이용해 100 크기의 데이터를 생성하기 위해 (1000,100) 크기의 데이터를 생성한다. requires_grad = True 로 설정해 Gradient를 계산할 수 있도록 설정한다.

(4) w2는 w1과 x를 행렬 곱한 결과에 계산할 수 있는 데이터이어야 한다. w1과 x의 행렬 곱한 결과는 (1,100)이며 (100,10) 행렬을 통해 Output을 계산할 수 있도록 w2의 모양을 설정한다. w2 역시 Back Propagation을 통해 업데이트해야 하는 대상이므로 requires_grad = True로 설정한다.

In [ ]:
lr = 1e-6    # 1
for t in range(1,501): # 2
  y_pred = x.mm(w1).clamp(min = 0).mm(w2) # 3

  loss = (y_pred - y).pow(2).sum() # 4
  if t % 100 == 0: 
    print('Iteration: ', t, '\t', 'Loss: ', loss.item()) # 5
  loss.backward() # 6

  with torch.no_grad(): # 7
    w1 -= lr * w1.grad # 8
    w2 -= lr * w2.grad # 9

    w1.grad.zero_() # 10
    w2.grad.zero_() # 11

Iteration:  100 	 Loss:  2.609075636428315e-06
Iteration:  200 	 Loss:  2.2003246158419643e-06
Iteration:  300 	 Loss:  1.8561638626124477e-06
Iteration:  400 	 Loss:  1.6445692381239496e-06
Iteration:  500 	 Loss:  1.4831061889708508e-06


(1) 파라미터를 업데이트할 때, gradient를 계산한 결과값에 lr만큼 곱한 값을 이용해 업데이트된다. 이를 Learning rate(학습률) 라고한다. Learning rate를 어떻게 설정하느냐에 따라 gradient 값에 따른 학습 정도가 결정된다.

(2) 500번 반복해 파라미터 값을 업데이트하기 위해 반복문 설정한다.

(3) 딥러닝 모델의 결과값을 보통 '예측값' 이라고 표현한다. 딥러닝 모델의 Input인 x와 parameter w1 간의 행렬 곱을 이용해 나온 결과값을 계산한다. 그 이후 torch 모듈 내 clamp라는 메서드를 이용해 비선형 함수를 적용. 여기서 clamp는 비선형 함수 ReLu()와 같은 역할을 한다. 최소값이 0이며 0보다 큰 자기 자신을 갖게 되는 매서드이기 때문이다. clamp를 이용해 계산된 결과와 w2를 이용해 행렬 곱을 한 번더 계산한다. 행렬 곱을 한 결과는 딥러닝 모델에서의 output을 의미하며 이는 예측값이라고 표현되기 때문에 y_pred으로 지정

(4) 예측값과 실제 레이블 값을 비교해 오차를 계산한 값을 loss(cost)라고 한다. 예측값을 의미하는 y_pred와 실제 레이블을 의미하는 y 간의 차잇값을 계산한 후 torch 모듈내에 pow(거듭제곱 함수) 함수를 이용해 제곱을 취한다. 즉 y_pred - y.pow(2) 는 제곱차를 의미하며 제곱차의 합을 sum()을 이용해 계산한다.

(5) 반복 100번 마다 현재 진행 중인 반복문 횟수와 Loss 값을 출력한다.

(6) 계산된 Loss 값에 대해 backward() 메서드를 이용하면 각 파라미터 값에 대해 gradient를 계산하고 이를 통해 back propagation을 진행한다는 것을 의미한다.

(7) 각 파라미터 값에 대해 gradient를 계산한 결과를 이용해 파라미터 값을 업데이트할 때는 해당 시점의 gradient 값을 고정한 후 업데이트를 진행한다. 코드가 실행되는 시점에서 gradient값을 고정한다는 의미

(8) grdient 값을 고정한 상태에서 w1의 gradient 값을 의미하는 w1.grad 에 (1)에서 설정한 lr 값을 곱한 결과값을 기존 w1에서 빼준다. 음수를 이용하는 이유는 loss 값이 최소로 계산될 수 있는 파라미터 값을 찾기 위해 gradient 값에 대한 반대 방향으로 계산한다는 것을 의미

(9) 8번과 동일

(10) , (11) 8번과 9번을 통해 각 파라미터 값을 업데이트했다면 각 파라미터 값의 gradien를 초기화해 다음 반복문을 진행할 수 있도록 gradient 값을 0으로 설정 다음 backpropagation 을 진행할 때 gradient 값을 loss.backward() 을 통해 새로 계산하기 때문이다.

500번의 반복을 실행하면서 loss 값이 줄어드는 것을 확인할 수 있다. loss값이 줄어든다는 것은 input이 w1과 w2를 통해 계산된 결과값이 y에 점점 비슷해진다는 것을 의미하며 y값과 비슷한 output을 계산할 수 있도록 w1과 w2가 계산된다는 것을 알 수 있다.